In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/MzeroMiko/VMamba

Cloning into 'VMamba'...
remote: Enumerating objects: 8812, done.
remote: Counting objects: 100% (1439/1439), done.
remote: Compressing objects: 100% (257/257), done.
remote: Total 8812 (delta 1332), reused 1227 (delta 1178), pack-reused 7373
Receiving objects: 100% (8812/8812), 34.78 MiB | 15.63 MiB/s, done.
Resolving deltas: 100% (5429/5429), done.


In [3]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 19.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

In [4]:
!pip install torchinfo

In [6]:
!pip install -U mamba-ssm causal-conv1d

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 7.5 MB/s eta 0:00:00
  Created wheel for mamba-ssm: filename=mamba_ssm-2.2.2-cp310-cp310-linux_x86_64.whl size=323803485 sha256=cd8ee941b25398d90c135d3a180b5dc33e478c98ae4998b61749809beaff947a
  Stored in directory: /root/.cache/pip/wheels/57/7c/90/9f963468ecc3791e36e388f9e7b4a4e1e3f90fbb340055aa4d
  Created wheel for causal-conv1d: filename=causal_conv1d-1.4.0-cp310-cp310-linux_x86_64.whl size=104684541 sha256=7406be22f70ba3bb956556f0f0ce04608b5ad00fc171df39b562c97dc757731d
  Stored in directory: /root/.cache/pip/wheels/e3/dd/4c/205f24e151736bd22f5980738dd10a19af6f093b6f4dcab006
Successfully built mamba-ssm causal-conv1d


In [7]:
!cd VMamba && pip install -r requirements.txt && cd ..

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=7409e44b07fe2704503e902060cc7783e6e73e91fb75d577fbeddc5f9d984acc
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532

In [13]:
!cd VMamba/kernels/selective_scan && pip install . && cd ../../..

Processing /content/VMamba/kernels/selective_scan
  Preparing metadata (setup.py) ... done
  Created wheel for selective-scan: filename=selective_scan-0.0.2-cp310-cp310-linux_x86_64.whl size=14235230 sha256=fd1818f31154feba7147ca8c6e15514cd7adb67480027b05c12516c355014284
  Stored in directory: /tmp/pip-ephem-wheel-cache-0w7zxz47/wheels/e2/41/fc/a434d222ee7f8f86faed67a5a74797c79f71177253ce354849
Successfully built selective-scan
  Attempting uninstall: selective-scan
    Found existing installation: selective-scan 0.0.2
    Uninstalling selective-scan-0.0.2:
      Successfully uninstalled selective-scan-0.0.2


In [1]:
!pip install -U torch

  Using cached torch-2.3.1-cp310-cp310-manylinux1_x86_64.whl (779.1 MB)
  Using cached triton-2.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (168.1 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.3.0
    Uninstalling triton-2.3.0:
      Successfully uninstalled triton-2.3.0
  Attempting uninstall: torch
    Found existing installation: torch 2.3.0
    Uninstalling torch-2.3.0:
      Successfully uninstalled torch-2.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 2.3.1 which is incompatible.
torchvision 0.18.0+cu121 requires torch==2.3.0, but you have torch 2.3.1 which is incompatible.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from albumentations.pytorch import ToTensorV2
import numpy as np
from tqdm import tqdm
from torch.utils.data.dataset import Dataset
from PIL import Image
import glob
import os
import torch.nn.functional as F
import albumentations as A
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from torchmetrics.classification import MulticlassJaccardIndex
import shutil
import cv2
import json
from einops.layers.torch import Rearrange, Reduce
from torch import Tensor
from einops import rearrange, repeat
from torchinfo import summary
from VMamba.classification.models.vmamba import VSSBlock

In [10]:
!wget https://landcover.ai.linuxpolska.com/download/landcover.ai.v1.zip

--2024-07-11 22:07:09--  https://landcover.ai.linuxpolska.com/download/landcover.ai.v1.zip
Resolving landcover.ai.linuxpolska.com (landcover.ai.linuxpolska.com)... 195.78.67.65
Connecting to landcover.ai.linuxpolska.com (landcover.ai.linuxpolska.com)|195.78.67.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1538212277 (1.4G) [application/zip]
Saving to: ‘landcover.ai.v1.zip’

landcover.ai.v1.zip 100%[===================>]   1.43G  14.6MB/s    in 1m 45s  

2024-07-11 22:08:55 (13.9 MB/s) - ‘landcover.ai.v1.zip’ saved [1538212277/1538212277]



In [11]:
!unzip landcover.ai.v1.zip -d landcover.ai.v1

Archive:  landcover.ai.v1.zip
   creating: landcover.ai.v1/images/
  inflating: landcover.ai.v1/images/M-33-48-A-c-4-4.tif  
  inflating: landcover.ai.v1/images/M-33-20-D-c-4-2.tif  
  inflating: landcover.ai.v1/images/M-33-20-D-d-3-3.tif  
  inflating: landcover.ai.v1/images/M-33-32-B-b-4-4.tif  
  inflating: landcover.ai.v1/images/M-33-7-A-d-2-3.tif  
  inflating: landcover.ai.v1/images/M-33-7-A-d-3-2.tif  
  inflating: landcover.ai.v1/images/M-34-32-B-a-4-3.tif  
  inflating: landcover.ai.v1/images/M-34-32-B-b-1-3.tif  
  inflating: landcover.ai.v1/images/M-34-5-D-d-4-2.tif  
  inflating: landcover.ai.v1/images/M-34-51-C-b-2-1.tif  
  inflating: landcover.ai.v1/images/M-34-51-C-d-4-1.tif  
  inflating: landcover.ai.v1/images/M-34-55-B-b-4-1.tif  
  inflating: landcover.ai.v1/images/M-34-56-A-b-1-4.tif  
  inflating: landcover.ai.v1/images/M-34-6-A-d-2-2.tif  
  inflating: landcover.ai.v1/images/M-34-65-D-a-4-4.tif  
  inflating: landcover.ai.v1/images/M-34-65-D-c-4-2.tif  
  inflati

In [12]:
IMGS_DIR = "./landcover.ai.v1/images"
MASKS_DIR = "./landcover.ai.v1/masks"
OUTPUT_DIR = "./landcover.ai.v1/output"

TARGET_SIZE = 512

img_paths = glob.glob(os.path.join(IMGS_DIR, "*.tif"))
mask_paths = glob.glob(os.path.join(MASKS_DIR, "*.tif"))

img_paths.sort()
mask_paths.sort()

os.makedirs(OUTPUT_DIR)
for i, (img_path, mask_path) in enumerate(zip(img_paths, mask_paths)):
    img_filename = os.path.splitext(os.path.basename(img_path))[0]
    mask_filename = os.path.splitext(os.path.basename(mask_path))[0]
    img = cv2.imread(img_path)
    mask = cv2.imread(mask_path)

    assert img_filename == mask_filename and img.shape[:2] == mask.shape[:2]

    k = 0
    for y in range(0, img.shape[0], TARGET_SIZE):
        for x in range(0, img.shape[1], TARGET_SIZE):
            img_tile = img[y : y + TARGET_SIZE, x : x + TARGET_SIZE]
            mask_tile = mask[y : y + TARGET_SIZE, x : x + TARGET_SIZE]

            if img_tile.shape[0] == TARGET_SIZE and img_tile.shape[1] == TARGET_SIZE:
                out_img_path = os.path.join(
                    OUTPUT_DIR, "{}_{}.tif".format(img_filename, k)
                )
                cv2.imwrite(out_img_path, img_tile)

                out_mask_path = os.path.join(
                    OUTPUT_DIR, "{}_{}_m.tif".format(mask_filename, k)
                )
                cv2.imwrite(out_mask_path, mask_tile)

            k += 1

    print("Processed {} {}/{}".format(img_filename, i + 1, len(img_paths)))

Processed M-33-20-D-c-4-2 1/41
Processed M-33-20-D-d-3-3 2/41
Processed M-33-32-B-b-4-4 3/41
Processed M-33-48-A-c-4-4 4/41
Processed M-33-7-A-d-2-3 5/41
Processed M-33-7-A-d-3-2 6/41
Processed M-34-32-B-a-4-3 7/41
Processed M-34-32-B-b-1-3 8/41
Processed M-34-5-D-d-4-2 9/41
Processed M-34-51-C-b-2-1 10/41
Processed M-34-51-C-d-4-1 11/41
Processed M-34-55-B-b-4-1 12/41
Processed M-34-56-A-b-1-4 13/41
Processed M-34-6-A-d-2-2 14/41
Processed M-34-65-D-a-4-4 15/41
Processed M-34-65-D-c-4-2 16/41
Processed M-34-65-D-d-4-1 17/41
Processed M-34-68-B-a-1-3 18/41
Processed M-34-77-B-c-2-3 19/41
Processed N-33-104-A-c-1-1 20/41
Processed N-33-119-C-c-3-3 21/41
Processed N-33-130-A-d-3-3 22/41
Processed N-33-130-A-d-4-4 23/41
Processed N-33-139-C-d-2-2 24/41
Processed N-33-139-C-d-2-4 25/41
Processed N-33-139-D-c-1-3 26/41
Processed N-33-60-D-c-4-2 27/41
Processed N-33-60-D-d-1-2 28/41
Processed N-33-96-D-d-1-1 29/41
Processed N-34-106-A-b-3-4 30/41
Processed N-34-106-A-c-1-3 31/41
Processed N-

In [13]:
train_data_dir = "landcover.ai.v1/train/image"
train_label_dir = "landcover.ai.v1/train/label"

val_data_dir = "landcover.ai.v1/val/image"
val_label_dir = "landcover.ai.v1/val/label"

test_data_dir = "landcover.ai.v1/test/image"
test_label_dir = "landcover.ai.v1/test/label"

paths = [
    train_data_dir,
    train_label_dir,
    val_data_dir,
    val_label_dir,
    test_data_dir,
    test_label_dir,
]

for p in paths:
    if not os.path.exists(p):
        os.makedirs(p)
        print(f"Folder '{p}' created.")
    else:
        print(f"Folder '{p}' already exists.")


with open("landcover.ai.v1/train.txt") as f:
    train_split = f.read().splitlines()

with open("landcover.ai.v1/val.txt") as f:
    val_split = f.read().splitlines()

with open("landcover.ai.v1/test.txt") as f:
    test_split = f.read().splitlines()

print("Train, Val, Test splits count")
print(len(train_split), len(val_split), len(test_split))
print("Last item of each split")
print(train_split[-1], val_split[-1], test_split[-1])

for i in train_split:
    source_file_img = os.path.join("landcover.ai.v1/output", f"{i}.tif")
    source_file_label = os.path.join("landcover.ai.v1/output", f"{i}_m.tif")
    destination_file_img = os.path.join(train_data_dir, f"{i}.tif")
    destination_file_label = os.path.join(train_label_dir, f"{i}.tif")
    if os.path.isfile(source_file_img) and not os.path.exists(destination_file_img):
        shutil.copy2(source_file_img, destination_file_img)
    if os.path.isfile(source_file_label) and not os.path.exists(destination_file_label):
        shutil.copy2(source_file_label, destination_file_label)

for i in val_split:
    source_file_img = os.path.join("landcover.ai.v1/output", f"{i}.tif")
    source_file_label = os.path.join("landcover.ai.v1/output", f"{i}_m.tif")
    destination_file_img = os.path.join(val_data_dir, f"{i}.tif")
    destination_file_label = os.path.join(val_label_dir, f"{i}.tif")
    if os.path.isfile(source_file_img) and not os.path.exists(destination_file_img):
        shutil.copy2(source_file_img, destination_file_img)
    if os.path.isfile(source_file_label) and not os.path.exists(destination_file_label):
        shutil.copy2(source_file_label, destination_file_label)

for i in test_split:
    source_file_img = os.path.join("landcover.ai.v1/output", f"{i}.tif")
    source_file_label = os.path.join("landcover.ai.v1/output", f"{i}_m.tif")
    destination_file_img = os.path.join(test_data_dir, f"{i}.tif")
    destination_file_label = os.path.join(test_label_dir, f"{i}.tif")
    if os.path.isfile(source_file_img) and not os.path.exists(destination_file_img):
        shutil.copy2(source_file_img, destination_file_img)
    if os.path.isfile(source_file_label) and not os.path.exists(destination_file_label):
        shutil.copy2(source_file_label, destination_file_label)

print("File copying completed.")

Folder 'landcover.ai.v1/train/image' created.
Folder 'landcover.ai.v1/train/label' created.
Folder 'landcover.ai.v1/val/image' created.
Folder 'landcover.ai.v1/val/label' created.
Folder 'landcover.ai.v1/test/image' created.
Folder 'landcover.ai.v1/test/label' created.
Train, Val, Test splits count
7470 1602 1602
Last item of each split
N-34-97-D-c-2-4_9 N-34-97-D-c-2-4_75 N-34-97-D-c-2-4_77
File copying completed.


In [3]:
torch.cuda.is_available()

True

In [4]:
class PatchEmbedding(nn.Module):
    def __init__(self):
        super().__init__()
        self.projection = nn.Sequential(
            nn.Conv2d(3, 1024, kernel_size=16, stride=16),
            Rearrange('b e (h) (w) -> b (h w) e'),
        )
        self.positions = nn.Parameter(torch.randn((512 // 16) ** 2, 1024))

    def forward(self, x: Tensor) -> Tensor:
        x = self.projection(x)
        x += self.positions
        return x

class TransformerBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.norm1 = nn.LayerNorm(1024)
        self.attn = nn.MultiheadAttention(1024, 8, batch_first=True)
        self.norm2 = nn.LayerNorm(1024)
        self.mlp = nn.Sequential(
            nn.Linear(1024, 1024 * 2),
            nn.GELU(),
            nn.Linear(1024 * 2, 1024)
        )

    def forward(self, x):
        x = x + self.attn(self.norm1(x), self.norm1(x), self.norm1(x))[0]
        x = x + self.mlp(self.norm2(x))
        return x

class SETRM(nn.Module):
    def __init__(self, num_transformer_layers, mamba_to_transformer_ratio):
        super().__init__()

        self.patch_embed = PatchEmbedding()

        self.num_transformer_layers = num_transformer_layers
        self.mamba_to_transformer_ratio = mamba_to_transformer_ratio

        self.transformer_layers = nn.ModuleList()
        self.mamba_layers = nn.ModuleList()

        for i in range(num_transformer_layers):
            self.transformer_layers.append(TransformerBlock())

        for i in range(num_transformer_layers * mamba_to_transformer_ratio):
            self.mamba_layers.append(VSSBlock(hidden_dim=1024, ssm_init="v0", forward_type="v0"))

        self.decoder_1 = nn.Sequential(
            nn.Conv2d(1024, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        )

        self.decoder_2 = nn.Sequential(
            nn.Conv2d(256, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        )

        self.decoder_3 = nn.Sequential(
            nn.Conv2d(128, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        )

        self.decoder_4 = nn.Sequential(
            nn.Conv2d(64, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        )

        self.final_out = nn.Conv2d(32, 5, 3, padding=1)

    def forward(self, x):
        x = self.patch_embed(x)

        for i in range(len(self.transformer_layers)):
            x = self.transformer_layers[i](x)
            if len(self.mamba_layers) > 0:
                for j in range(i * self.mamba_to_transformer_ratio, (i * self.mamba_to_transformer_ratio) + self.mamba_to_transformer_ratio):
                    x = rearrange(x, "b (h w) c -> b h w c", h = 512 // 16)
                    x = self.mamba_layers[j](x)
                    x = rearrange(x, 'b (h) (w) c -> b (h w) c')

        x = rearrange(x, "b (h w) c -> b c h w", h = 512 // 16)

        x = self.decoder_1(x)
        x = self.decoder_2(x)
        x = self.decoder_3(x)
        x = self.decoder_4(x)

        x = self.final_out(x)

        return x

In [5]:
def preprocess_data(image, mask, train):
    if train:
        image_transforms = A.Compose(
            [
                A.OneOf([
                    A.ToGray(),
                    A.HueSaturationValue(hue_shift_limit=3, sat_shift_limit=3, val_shift_limit=3),
                    A.RandomBrightnessContrast(brightness_limit=0.01, contrast_limit=0.01, brightness_by_max=False)
                ], p=0.2),
                A.Normalize(),
                ToTensorV2()
            ]
        )
    else:
        image_transforms = A.Compose(
            [
                A.Normalize(),
                ToTensorV2()
            ]
        )

    image = image_transforms(image=image)
    image = image['image']

    mask = torch.from_numpy(mask)
    mask = torch.permute(mask, (2, 0, 1))
    mask = mask[0]

    return image, mask

In [6]:
class CustomDataset(Dataset):
    def __init__(self, image_paths, target_paths, preprocess_fn, train=False):
        self.image_paths = image_paths
        self.target_paths = target_paths
        self.preprocess_fn = preprocess_fn
        self.train = train

    def __getitem__(self, index):
        image = Image.open(self.image_paths[index])
        mask = Image.open(self.target_paths[index])

        return self.preprocess_fn(np.array(image), np.array(mask, dtype=np.int8), train=self.train)

    def __len__(self):
        return len(self.image_paths)

In [7]:
class DiceLoss(nn.Module):
    """Dice Loss PyTorch
        Created by: Zhang Shuai
        Email: shuaizzz666@gmail.com
        dice_loss = 1 - 2*p*t / (p^2 + t^2). p and t represent predict and target.
    Args:
        weight: An array of shape [C,]
        predict: A float32 tensor of shape [N, C, *], for Semantic segmentation task is [N, C, H, W]
        target: A int64 tensor of shape [N, *], for Semantic segmentation task is [N, H, W]
    Return:
        diceloss
    """
    def __init__(self, weight=None):
        super(DiceLoss, self).__init__()
        if weight is not None:
            weight = torch.Tensor(weight)
            self.weight = weight / torch.sum(weight) # Normalized weight
        self.smooth = 1e-5

    def forward(self, predict, target):
        N, C = predict.size()[:2]
        predict = predict.view(N, C, -1) # (N, C, *)
        target = target.view(N, 1, -1) # (N, 1, *)

        predict = F.softmax(predict, dim=1) # (N, C, *) ==> (N, C, *)
        ## convert target(N, 1, *) into one hot vector (N, C, *)
        target_onehot = torch.zeros(predict.size()).cuda()  # (N, 1, *) ==> (N, C, *)
        target_onehot.scatter_(1, target, 1)  # (N, C, *)

        intersection = torch.sum(predict * target_onehot, dim=2)  # (N, C)
        union = torch.sum(predict.pow(2), dim=2) + torch.sum(target_onehot, dim=2)  # (N, C)
        ## p^2 + t^2 >= 2*p*t, target_onehot^2 == target_onehot
        dice_coef = (2 * intersection + self.smooth) / (union + self.smooth)  # (N, C)

        if hasattr(self, 'weight'):
            if self.weight.type() != predict.type():
                self.weight = self.weight.type_as(predict)
                dice_coef = dice_coef * self.weight * C  # (N, C)
        dice_loss = 1 - torch.mean(dice_coef)  # 1

        return dice_loss

In [8]:
class CEPlusDiceLoss(nn.Module):
    def __init__(self):
        super(CEPlusDiceLoss, self).__init__()
        self.dice_loss = DiceLoss()
        self.ce_loss = nn.CrossEntropyLoss()

    def forward(self, predict, target):
        dice = self.dice_loss(predict, target.to(torch.int64))
        ce = self.ce_loss(predict, target.long())

        total = dice + ce

        return total

In [9]:
batch_size = 16
num_epochs = 5
num_classes = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
show_summary = False

In [10]:
train_img_paths = glob.glob(os.path.join("landcover.ai.v1/train/image", "*.tif"))
train_img_paths = sorted(train_img_paths)

train_mask_paths = glob.glob(os.path.join("landcover.ai.v1/train/label", "*.tif"))
train_mask_paths = sorted(train_mask_paths)

val_img_paths = glob.glob(os.path.join("landcover.ai.v1/val/image", "*.tif"))
val_img_paths = sorted(val_img_paths)

val_mask_paths = glob.glob(os.path.join("landcover.ai.v1/val/label", "*.tif"))
val_mask_paths = sorted(val_mask_paths)

In [11]:
train_dataset = CustomDataset(
    image_paths=train_img_paths,
    target_paths=train_mask_paths,
    preprocess_fn=preprocess_data,
    train=True
)
val_dataset = CustomDataset(
    image_paths=val_img_paths,
    target_paths=val_mask_paths,
    preprocess_fn=preprocess_data,
)

train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)
val_loader = DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False
)

In [12]:
model = SETRM(num_transformer_layers=2, mamba_to_transformer_ratio=1).to(device)
loss_fn = CEPlusDiceLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
metric = MulticlassJaccardIndex(num_classes=num_classes).to(device)

In [12]:
if show_summary:
    print(summary(model, input_size=(1, 3, 512, 512)))

In [13]:
epoch_train_loss = []
epoch_train_iou = []
epoch_val_loss = []
epoch_val_iou = []

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}:")

    model.train()
    train_loss = 0
    train_iou = 0

    for images, masks in tqdm(train_loader, desc="Training"):
        images, masks = images.to(device), masks.to(device)
        outputs = model(images)
        loss = loss_fn(outputs, masks)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_iou += metric(outputs, masks)

    train_loss = train_loss / len(train_loader)
    train_iou = train_iou / len(train_loader)

    epoch_train_loss.append(train_loss)
    epoch_train_iou.append(train_iou)

    print(f"Train Loss: {train_loss:.4f}")
    print(f"Train IoU: {train_iou:.4f}")

    model.eval()
    val_loss = 0
    val_iou = 0

    with torch.inference_mode():
        for images, masks in tqdm(val_loader, desc="Validation"):
            images, masks = images.to(device), masks.to(device)

            outputs = model(images)
            loss = loss_fn(outputs, masks)

            val_loss += loss.item()
            val_iou += metric(outputs, masks)

    val_loss = val_loss / len(val_loader)
    val_iou = val_iou / len(val_loader)

    epoch_val_loss.append(val_loss)
    epoch_val_iou.append(val_iou)

    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Validation IoU: {val_iou:.4f}")
    print("-----------------------------")

Epoch 1/5:


Training: 100%|██████████| 467/467 [11:41<00:00,  1.50s/it]


Train Loss: 1.5093
Train IoU: 0.2541


Validation: 100%|██████████| 101/101 [01:11<00:00,  1.42it/s]


Validation Loss: 1.4067
Validation IoU: 0.2968
-----------------------------
Epoch 2/5:


...

Validation Loss: 1.3125
Validation IoU: 0.3111
-----------------------------
Epoch 5/5:


Training: 100%|██████████| 467/467 [11:34<00:00,  1.49s/it]


Train Loss: 1.4507
Train IoU: 0.2781


Validation: 100%|██████████| 101/101 [01:09<00:00,  1.46it/s]

Validation Loss: 1.7215
Validation IoU: 0.1951
-----------------------------


In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/landcover_seg_model.pth")
torch.save(optimizer.state_dict(), "/content/drive/MyDrive/landcover_seg_opt.pth")

In [ ]:
training_stats = {
    "train_loss": epoch_train_loss,
    "train_iou": [el.item() for el in epoch_train_iou],
    "val_loss": epoch_val_loss,
    "val_iou": [el.item() for el in epoch_val_iou]
}

with open("/content/drive/MyDrive/mrp-train-stats.json", "w") as outfile:
    json.dump(training_stats, outfile)

In [ ]:
from google.colab import runtime
runtime.unassign()